# Naselaris Lab fMRI Preprocessing Pipeline

In [5]:
from fsl_preproc import create_fsl_preproc_workflow
from mriDbObjDefs_pandas import *
import time
import sys

In [6]:
main_inputnode = fsl_preproc_inode()  # in mriDbObjDefs
pipeline_data_params = dict()  # Select only...
pipeline_data_params['subject'] = 'TN'
pipeline_data_params['experiment'] = 'lotusHill.7T.em9K.1.5iso'
pipeline_data_params['experiment'] = '3T.v.7T'
pipeline_data_params['run_list'] = []
pipeline_data_params['sess_list'] = []
pipeline_data_params['crop_this'] = ['med', 'med', 'min']  # Almost never used
pipeline_data_params['db'] = '/mnt/em9k.csv' # this can be a csv or a pickled pandas DataFrame

In [7]:
main_inputnode.panda_fields(
    subj = pipeline_data_params['subject'],
    expID = pipeline_data_params['experiment'],
    runList = pipeline_data_params['run_list'],
    sessList=pipeline_data_params['sess_list'],
    cropRule = pipeline_data_params['crop_this'],
    db = pipeline_data_params['db'],

    )

# checking the data

In [15]:
main_inputnode.DF[main_inputnode.DF.subject == pipeline_data_params['subject']].tail(5)

,runID,date,subject,expID,working_vol,brain_mask,sessionID,runType,design matrix,frame_file,...,vox_x,vox_y,vox_z,run_code_path,run_code_file,design_matrix_path,frame_file_path,pic_path,siemensRef,padVol
66,203,11/16/2011,TN,lotusHill.7T.em9K.1.5iso,/mnt/initial_em9k_pipe/TN_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/TN_nofnirt//lotusHill.7...,19,trn,onoffTrn.mat,trn018.txt,...,1.5,1.5,1.5,/musc.repo/mri/7T.cmrr/Presentation/buffermovi...,Nov.16.2011.7T.TN.py,/musc.repo/mri/7T.cmrr/Presentation/Nov2011,/musc.repo/mri/7T.cmrr/Presentation/Nov2011,/musc.repo/mri/7T.cmrr/Presentation/pictures,1,0
67,204,11/16/2011,TN,lotusHill.7T.em9K.1.5iso,/mnt/initial_em9k_pipe/TN_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/TN_nofnirt//lotusHill.7...,19,val,onoffVal.mat,val007.txt,...,1.5,1.5,1.5,/musc.repo/mri/7T.cmrr/Presentation/buffermovi...,Nov.16.2011.7T.TN.py,/musc.repo/mri/7T.cmrr/Presentation/Nov2011,/musc.repo/mri/7T.cmrr/Presentation/Nov2011,/musc.repo/mri/7T.cmrr/Presentation/pictures,1,0
68,205,11/16/2011,TN,lotusHill.7T.em9K.1.5iso,/mnt/initial_em9k_pipe/TN_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/TN_nofnirt//lotusHill.7...,19,trn,onoffTrn.mat,trn019.txt,...,1.5,1.5,1.5,/musc.repo/mri/7T.cmrr/Presentation/buffermovi...,Nov.16.2011.7T.TN.py,/musc.repo/mri/7T.cmrr/Presentation/Nov2011,/musc.repo/mri/7T.cmrr/Presentation/Nov2011,/musc.repo/mri/7T.cmrr/Presentation/pictures,1,0
69,206,11/16/2011,TN,lotusHill.7T.em9K.1.5iso,/mnt/initial_em9k_pipe/TN_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/TN_nofnirt//lotusHill.7...,19,imgVal,onoffImg.mat,imgVal.txt,...,1.5,1.5,1.5,/musc.repo/mri/7T.cmrr/Presentation/buffermovi...,Nov.16.2011.7T.TN.py,/musc.repo/mri/7T.cmrr/Presentation/Nov2011,/musc.repo/mri/7T.cmrr/Presentation/Nov2011,/musc.repo/mri/7T.cmrr/Presentation/pictures,1,0
70,207,11/16/2011,TN,lotusHill.7T.em9K.1.5iso,/mnt/initial_em9k_pipe/TN_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/TN_nofnirt//lotusHill.7...,19,img,onoffImg.mat,img.txt,...,1.5,1.5,1.5,/musc.repo/mri/7T.cmrr/Presentation/buffermovi...,Nov.16.2011.7T.TN.py,/musc.repo/mri/7T.cmrr/Presentation/Nov2011,/musc.repo/mri/7T.cmrr/Presentation/Nov2011,/musc.repo/mri/7T.cmrr/Presentation/pictures,1,0


In [17]:
fsl_preproc_params = dict()
fsl_preproc_params['basedir'] = '/mnt/nipype_intermediate'
fsl_preproc_params['results_base_dir'] = '/mnt/pipeline_results/'
fsl_preproc_params['results_container'] = 'em9k'
fsl_preproc_params['convert_dicoms'] = False
fsl_preproc_params['t_size'] = 10000  # just go with it (max possible size).
fsl_preproc_params['bet_frac'] = 0.08  # BRAIN EXTRACTION TOOL THRESHOLD
# motion correction
fsl_preproc_params['ref_run'] = []  # reference volume all others will be referenced
fsl_preproc_params['moco_only'] = False  # if true, all FLIRT AND FNIRT is ignored
# linear registration#
fsl_preproc_params['FLIRT_cost_func'] = ['normcorr']
fsl_preproc_params['interp_FLIRT'] = 'sinc'  # linear registration stage (sinc, trilinear, nearestneighbour, spline)
fsl_preproc_params['dof_FLIRT'] = 6  # degrees of freedom (MAX : 12)
fsl_preproc_params['rigid2D_FLIRT'] = False  # if true, restrict linear reg to rigid body transformations and ignore dof
# nonlinear registration
fsl_preproc_params['do_FNIRT'] = True
fsl_preproc_params['searchr_x'] = []  # params for linear reg exposed (select the angular range over which the initial optimisation search stage is performed.)
fsl_preproc_params['searchr_y'] = []  # find out what these guys do
fsl_preproc_params['searchr_z'] = []
fsl_preproc_params['interp_FNIRT'] = 'spline'  # what kind of interpolation (sinc,trilinear,nearestneighbour or spline)
fsl_preproc_params['FNIRT_subsamp'] = [[4, 2, 1, 1]]  # FNIRT runs a coarse-to-fine algorithm. This is a list specifying the downsampling factor on each iteration.
fsl_preproc_params['FNIRT_warpres'] = [(5, 5, 5)]  # Resolution of the warping function. Like, how fine is the warping. Can specify different level for each iteration. *Question*: Why is this list shorter than the one above?
# run settings
fsl_preproc_params['nProc'] = 1 # number of CPUS

# dump params in input node
for key in fsl_preproc_params.keys():
    setattr(main_inputnode.inputs, key, fsl_preproc_params[key])

# => construct the workflow
pp = create_fsl_preproc_workflow(main_inputnode)
pp.base_dir = fsl_preproc_params['basedir']

# print inputs after being processed by mriDbObjDefs for sanity check
print main_inputnode.inputs


FLIRT_cost_func = ['normcorr']
FNIRT_subsamp = [[4, 2, 1, 1]]
FNIRT_warpres = [(5, 5, 5)]
abs_run_id = [160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207]
basedir = /mnt/nipype_intermediate
bet_frac = 0.08
convert_dicoms = False
do_FNIRT = True
dof_FLIRT = 6
interp_FLIRT = sinc
interp_FNIRT = spline
mags = <undefined>
moco_only = False
nProc = 1
nVols = [265, 265, 265, 265, 236, 236, 265, 265, 265, 265, 265, 265, 265, 265, 265, 236, 236, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 236, 236]
padNum = 0
phase = <undefined>
ref_run = []
results_base_dir = /mnt/pipeline_results/
results_container = em9k
rigid2D_FLIRT = False
rigid2D_FLIRTconvert_dicoms = <undefined>
run_list = ['/mnt/vison_conv/naselaris_TN/1_006_eja_ep2d_bold_1_5mm_2000TR_FH_20111107/naselaris_TN_20111107_001_006_eja_ep2d_bold_1_5mm_2000TR_FH.nii', '/mnt/vison_conv/naselaris_TN/1_008

/home/nick/anaconda/lib/python2.7/site-packages/nipype/interfaces/base.py:397: UserWarning: Input concat_xfm requires inputs: in_file2
  warn(msg)


# run the pipeline

In [ ]:
start = time.time()
if fsl_preproc_params['nProc'] > 1:
    pp.run(plugin='MultiProc', plugin_args={'n_procs': fsl_preproc_params['nProc']})
else:
    pp.run()
last_words = "Pipeline ran in {} hours on {} items"
print last_words.format(time.strftime("%H:%M:%S", time.gmtime(time.time() - start)), len(main_inputnode.inputs.abs_run_id))

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node dropref in dir: /mnt/nipype_intermediate/fsl_preproc/killref/dropref
INFO:workflow:Executing node _dropref0 in dir: /mnt/nipype_intermediate/fsl_preproc/killref/dropref/mapflow/_dropref0
INFO:workflow:Running: fslroi /mnt/vison_conv/naselaris_TN/1_006_eja_ep2d_bold_1_5mm_2000TR_FH_20111107/naselaris_TN_20111107_001_006_eja_ep2d_bold_1_5mm_2000TR_FH.nii /mnt/nipype_intermediate/fsl_preproc/killref/dropref/mapflow/_dropref0/naselaris_TN_20111107_001_006_eja_ep2d_bold_1_5mm_2000TR_FH_roi.nii.gz 0 108 0 108 0 39 1 10000
INFO:workflow:Executing node _dropref1 in dir: /mnt/nipype_intermediate/fsl_preproc/killref/dropref/mapflow/_dropref1
INFO:workflow:Running: fslroi /mnt/vison_conv/naselaris_TN/1_008_eja_ep2d_bold_1_5mm_2000TR_FH_trn2_20111107/naselaris_TN_20111107_001_008_eja_ep2d_bold_1_5mm_2000TR_FH_trn2.nii /mnt/nipype_intermediate/fsl_preproc/killref/dropref/mapflow/_dropref1/na

# Appending Working Volume and Brain Mask file paths to a new CSV which is saved in results_base_dir


In [ ]:
add_mask_vols(pp.get_node('inputnode'), fsl_preproc_params, pipeline_data_params['db'])
# outputs a text file of parameters used

# Saves a txt file in results_base_dir with parameters used

In [ ]:
params_txt(fsl_preproc_params)